In [1]:
import json
import httpx
import asyncio
import pandas as pd

from os import path
from toolkit.models import Timeframe
from toolkit.exchanges import Binance, Coinbase

In [2]:
timeframe = Timeframe.MINUTLY
exchanges = {
    "Binance": {
        "client": Binance(),
        "markets": ["BTCUSDT", "ETHUSDT", "LTCUSDT", "SOLUSDT", "ATOMUSDT", "XRPUSDT"],
        "kline": []
    },
    "Coinbase": {
        "client": Coinbase(),
        "markets": ["BTC-USD", "ETH-USD", "LTC-USD", "SOL-USD", "ATOM-USD", "XRP-USD"],
        "kline": []
    }
}

print("Downloading markets...")
async with httpx.AsyncClient() as client:
    for _, para in exchanges.items():
        for symbol in para["markets"]:
            para["kline"].append(para["client"].kline(client, symbol, timeframe))
        para["kline"] = await asyncio.gather(*para["kline"])
        para["kline"] = dict(zip(para["markets"], para["kline"]))

        for _, data in para["kline"].items():
            data.sort(key=lambda x: x.time)
print("Finished downloading.")

Finished downloading.


In [3]:
data_path = path.join("data")
index_path = path.join(data_path, "_index.json")

index = {}
if path.isfile(index_path):
    with open(index_path, "r") as f:
        index = json.load(f)

if len(index) == 0:
    for exchange, para in exchanges.items():
        index[exchange] = {}
        for symbol, trades in para["kline"].items():
            df = pd.DataFrame.from_dict(trades)
            index[exchange][symbol] = df.iloc[-1]["time"].isoformat()

            df.to_csv(path.join(data_path, f"{exchange}_{symbol}_Kline_{timeframe.value}.csv"), index=False)
    with open(index_path, "w+") as f:
        json.dump(index, f)
else:
    for exchange, para in exchanges.items():
        if exchange not in index:
            index[exchange] = {}

        for symbol, trades in para["kline"].items():
            df = pd.DataFrame.from_dict(trades)
            new = df.copy()
            
            if symbol in index[exchange]:
                new = df[df["time"] > index[exchange][symbol]]

            if not new.empty:
                new.to_csv(path.join(data_path, f"{exchange}_{symbol}_Kline_{timeframe.value}.csv"), mode="a", index=False, header=False)
                index[exchange][symbol] = new.iloc[-1]["time"].isoformat()
    with open(index_path, "w") as f:
        json.dump(index, f)